In [1]:
load('Imports/Utility.sage', 'Imports/Point.sage', 'Imports/Line.sage', 'Imports/TritangentPlane.sage', 'Imports/Group.sage', 'Imports/Cubic.sage')
import multiprocessing as mp
import pickle

In [2]:
(I-sqrt(3)).minpoly()

x^4 - 4*x^2 + 16

In [4]:
var('nv')
K1.<i_minus_rad3> = NumberField(nv^4-4*nv^2+16)

In [25]:
i = i_minus_rad3^3/8
rad3 = (i_minus_rad3^3 - 8*i_minus_rad3)/8

In [5]:
P.<x,y,z,t,b,c,d,e,f,l,m> = PolynomialRing(K1)
cubic_new = e*f*(2*x^2*y-2*x*y^2+x*z^2-x*z*t-y*t^2+y*z*t)+b*c*(x-t)*(x*z+y*t)+c*c*(z+t)*(y*t-x*z)+d*c*(y-z)*(x*z+y*t)+(e+f)*c*(x-y)*(y*t-x*z)
sing_cubics = (-1) * (-c + f) * (-c + e) * c * (c + f) * (c + e) * (-e + f)^2 * (-c*d + c*f + e*f) * (-c*d + c*e + e*f) * (-c^2 - c*d + e*f) * (b*c - c*f + e*f) * (b*c - c*e + e*f) * (b*c - c*d + 2*e*f) * (b*c - c^2 + e*f) * (b*c^2 + c^2*d + b*c*f - 2*c^2*f - c*d*f + 2*e*f^2) * (b*c^2 + c^2*d + b*c*e - 2*c^2*e - c*d*e + 2*e^2*f) * (-b*c^3 - 2*b*c^2*d + c^3*d + b*c^2*e + c^2*d*e + b*c^2*f + c^2*d*f + 3*b*c*e*f - 4*c^2*e*f - 3*c*d*e*f + 4*e^2*f^2)

In [6]:
# Define cubic SE9, find and classify the lines and compute the 45 tritangent planes

In [26]:
line = Line([y, z])
general_cubic = Cubic(cubic_new, line, sing_cubics)
SE9 = general_cubic.subs({b:-(c*c+e*f)/c, d:(3*e*f - c^2 + c*f + c*e)/(2*c)}).subs({e : i*rad3*c})

In [8]:
# Find simmetries and study possible subfamilies

In [27]:
%time adm_SE9 = SE9.find_admissible_projectivities()

CPU times: user 10.4 s, sys: 396 ms, total: 10.8 s
Wall time: 1min 32s


In [28]:
%time simm_SE9 = SE9.find_simmetries(adm_SE9)

CPU times: user 9.72 s, sys: 292 ms, total: 10 s
Wall time: 12.6 s


In [11]:
%time conds = SE9.find_conditions_for_subfamilies(adm_SE9, simm_SE9)
conds

CPU times: user 3min 38s, sys: 1.43 s, total: 3min 40s
Wall time: 3min 43s


[[52*c + (i_minus_rad3^3 - 4*i_minus_rad3^2 + 12*i_minus_rad3 + 4)*f],
 [52*c + (-i_minus_rad3^3 - 4*i_minus_rad3^2 - 12*i_minus_rad3 + 4)*f],
 [4*c + (i_minus_rad3^3 + 2*i_minus_rad3^2 - 4)*f],
 [13*c + (i_minus_rad3^3 - i_minus_rad3^2 - 3*i_minus_rad3 + 3)*f],
 [13*c + (-i_minus_rad3^3 - i_minus_rad3^2 + 3*i_minus_rad3 + 3)*f],
 [4*c + (-i_minus_rad3^3 + 2*i_minus_rad3^2 - 4)*f]]

In [12]:
# Choose one subfamily (say SE9') and study it

In [29]:
sol = solve_linear_system(conds[1], [c], [f])

In [30]:
SE9_1 = SE9.subs({c:sol[0], f:sol[1]})
print(SE9_1)
print(len(SE9_1.eckardt_points))

(-208*i_minus_rad3^2 + 416)*x^2*y + (208*i_minus_rad3^2 - 416)*x*y^2 + (24*i_minus_rad3^3 + 96*i_minus_rad3^2 - 128*i_minus_rad3 - 512)*x^2*z + (-44*i_minus_rad3^3 - 176*i_minus_rad3^2 + 96*i_minus_rad3 + 800)*x*y*z + (12*i_minus_rad3^3 + 48*i_minus_rad3^2 - 64*i_minus_rad3 - 256)*x*z^2 + (-32*i_minus_rad3^3 + 80*i_minus_rad3^2 + 32*i_minus_rad3 + 128)*x*y*t + (12*i_minus_rad3^3 - 160*i_minus_rad3^2 - 64*i_minus_rad3 + 160)*y^2*t + (20*i_minus_rad3^3 + 80*i_minus_rad3^2 + 32*i_minus_rad3 - 288)*y*z*t + (8*i_minus_rad3^3 + 32*i_minus_rad3^2 + 96*i_minus_rad3 - 32)*y*t^2
9


In [31]:
%time adm_SE9_1 = SE9_1.find_admissible_projectivities()
len(adm_SE9_1)

CPU times: user 4.84 s, sys: 228 ms, total: 5.06 s
Wall time: 43 s


1296

In [32]:
simm_SE9_1 = SE9_1.find_simmetries(adm_SE9_1)
len(simm_SE9_1)

108

In [17]:
# Study how the simmetries permute the Eckardt points

In [18]:
Eck_points_perms = []
for proj in simm_SE9_1:
    Eck_points_perms.append(Permutation(SE9_1.apply_proj_to_eck(proj)).to_permutation_group_element())
Eck_points_perms_group = PermutationGroup(Eck_points_perms)
Eck_points_perms_group.order(), Eck_points_perms_group.structure_description()

(36, '(C3 x C3) : C4')

In [19]:
# Study how the simmetries permute the 27 lines

In [20]:
lines_perms = []
for simm in simm_SE9_1:
    lines_perms.append(Permutation(SE9_1.apply_proj_to_lines(simm)).to_permutation_group_element())
lines_perms_group = PermutationGroup(lines_perms)
lines_perms_group.order(), lines_perms_group.structure_description()

(108, '((C3 x C3) : C3) : C4')